# Variational Autoencoder

Variational Autoencoders (VAEs) are dimensionality reduction devices that code inputs into the latent parameters of a statistical process that supposedly gave rise to them. In practice, this is typically a normal distribution. The decoding involves sampling from the statistical distribution with a given latent parameter and chasing the result through a decoder network.

The difference to classical autoencoders is that variational autoencoders map an input to a function (the distribution) and classical autoencoders map an input to a vector. 

c.f. this exellent post https://github.com/yaniv256/VAEs-in-Economics/blob/master/Notebooks/One_Dimensional_VAE_Workshop.ipynb

which uses Chapter 8 in http://faculty.neu.edu.cn/yury/AAI/Textbook/Deep%20Learning%20with%20Python.pdf

In [1]:
import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


def gpu_test():
    print("GPU test")
    with tf.device('GPU:0'):
        print("Executing basic TF graph on GPU:0")
        a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
        b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
        c = tf.matmul(a, b)
        print("Success.")
        print(c)
        print("GPUs found:")
        physical_devices = tf.config.experimental.list_physical_devices('GPU')
        print('\n'.join('[%i]\t'%i +str(dev) for i,dev in enumerate(physical_devices)))

gpu_test()

GPU test
Executing basic TF graph on GPU:0
Success.
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
GPUs found:
[0]	PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


from src.resourcelims import memory

"""
Check GPU 
"""
from tensorflow.python.client import device_lib
import tensorflow as tf
from keras import backend

#tf.debugging.set_log_device_placement(True)
#tf.config.experimental_run_functions_eagerly(True)
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)



@memory(0.8)
def gpu_test():
    
    print("TENSORFLOW")
    with tf.device('GPU:0'):
        
        a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
        b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
        c = tf.matmul(a, b)
        
    print(c)
    #print("LOCAL DEVICES",device_lib.list_local_devices())
    print("PHYSICAL DEVICES")
    print('\n'.join([str(x) for x in tf.config.list_physical_devices()]))

    #print(backend.tensorflow_backend._get_available_gpus())

gpu_test()

In [2]:
import keras
from keras import layers
from keras import backend as K 
from keras.models import Model 
from keras.datasets import mnist

import numpy as np
import tensorflow as tf


img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2

#@memory(0.8)
def make_model():

    with tf.device('GPU:0'):
        """
        Encoder
        """
        input_img = keras.Input(shape=img_shape)
        
        x = layers.Conv2D(16, 3, padding='same', activation='relu')(input_img)
        x = layers.Conv2D(32, 3, padding='same', activation='relu', strides=(2, 2))(x)
        x = layers.Conv2D(32, 3, padding='same', activation='relu')(x) 
        x = layers.Conv2D(32, 3, padding='same', activation='relu')(x) 

        #x = layers.Conv2D(32, 3, padding='same', activation='relu')(input_img)
        #x = layers.Conv2D(64, 3, padding='same', activation='relu', strides=(2, 2))(x) 
        #x = layers.Conv2D(64, 3, padding='same', activation='relu')(x) 
        #x = layers.Conv2D(64, 3, padding='same', activation='relu')(x) 

        shape_before_flattening = K.int_shape(x)

        x = layers.Flatten()(x)
        
        #x = layers.Dense(32, activation='relu')(x)
        x = layers.Dense(16, activation='relu')(x)

        # encoder output (latent parameters)
        z_mean = layers.Dense(latent_dim)(x) 
        z_log_var = layers.Dense(latent_dim)(x)


        """
        Sampling
        """
        def sampling(args):
            z_mean, z_log_var = args
            epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
            mean=0., stddev=1.) 
            return z_mean + K.exp(z_log_var) * epsilon

        # sample from latent distribution
        z = layers.Lambda(sampling)([z_mean, z_log_var])


        """
        Decoder
        """
        decoder_input = layers.Input(K.int_shape(z)[1:])
        x = layers.Dense(np.prod(shape_before_flattening[1:]), activation='relu')(decoder_input)
        x = layers.Reshape(shape_before_flattening[1:])(x)
        x = layers.Conv2DTranspose(32, 3, padding='same', activation='relu', strides=(2, 2))(x)
        x = layers.Conv2D(1, 3, padding='same', activation='sigmoid')(x)

        decoder = Model(decoder_input, x)

        # decoder output
        z_decoded = decoder(z)


        """
        Custom Loss
        """

        class CustomVariationalLayer(keras.layers.Layer):

            def vae_loss(self, x, z_decoded): 
                x = K.flatten(x)
                z_decoded = K.flatten(z_decoded)

                # reconstruction loss
                xent_loss = keras.metrics.binary_crossentropy(x, z_decoded) 

                # regularization loss
                kl_loss = -5e-4 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1) 

                return K.mean(xent_loss + kl_loss)

            def call(self, inputs):
                x = inputs[0]
                z_decoded = inputs[1]
                loss = self.vae_loss(x, z_decoded) 
                self.add_loss(loss, inputs=inputs) 
                return x

        # loss 
        y = CustomVariationalLayer()([input_img, z_decoded])


        """
        Training (on MNIST)
        """

        

        # compile model (input; loss)
        vae = Model(input_img, y) 
        vae.compile(optimizer='rmsprop', loss=None) 
        
        return vae

vae = make_model()        
vae.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 16)   160         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 14, 14, 32)   4640        conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 14, 14, 32)   9248        conv2d_1[0][0]                   
_______________________________________________________________________________________

In [ ]:
from src.resourcelims import memory

tf.config.run_functions_eagerly(True)

#@memory(0.90)
def train():
    with tf.device('GPU:0'):
        (x_train, _), (x_test, y_test) = mnist.load_data()

        x_train = x_train.astype('float32') / 255. 
        x_train = x_train.reshape(x_train.shape + (1,)) 

        x_test = x_test.astype('float32') / 255.
        x_test = x_test.reshape(x_test.shape + (1,))

        vae.fit(x=x_train, y=None, shuffle=True,
                epochs=10,
                batch_size=batch_size,
                validation_data=(x_test, None))
    return vae

vae = train()

/home/jpbm/.pyenv/versions/3.8.5/envs/occrp/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


Epoch 1/10
 338/3750 [=>............................] - ETA: 11:22 - loss: 0.0000e+00

This error is due to incompatibility in cuda, cudnn and Nvidia drivers or memory growth issue. The memory growth issue should be addressed so it's the former issues, most likely. TO DO.

In [ ]:
8*32*28*28